In [1]:
using WordNet
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Distances
using Iterators

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/DataStructures.ji for module DataStructures.
INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/StatsFuns.ji for module StatsFuns.
INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/Distances.ji for module Distances.
INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/NearestNeighbors.ji for module NearestNeighbors.
INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/Devectorize.ji for module Devectorize.
  likely near /home/ubuntu/.julia/v0.5/Devectorize/src/texpr.jl:293
  likely near /home/ubuntu/.julia/v0.5/Devectorize/src/texpr.jl:293
  likely near /home/ubuntu/.julia/v0.5/NearestNeighbors/src/knn.jl:1
  likely near /home/

In [4]:
const WN_PATH="data/corpora/WordNet-2.1/"
db = DB(WN_PATH)

WordNet.DB

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [3]:
#ee = restore("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.model");
ee = restore("models/ss/keep/tokenised_lowercase_WestburyLab.wikicorp.201004_50__m170000000.model");

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in utf8(::SubString{String}) at ./deprecated.jl:50
 in _collect(::Array{SubString{String},1}, ::Base.Generator{Array{SubString{String},1},Base.#utf8}, ::Base.EltypeUnknown, ::Base.HasShape) at ./array.jl:283
 in #error_content#12(::Symbol, ::String, ::Function, ::LoadError, ::Array{Ptr{Void},1}) at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:75
 in execute_request_0x535c5df2(::ZMQ.Socket, ::IJulia.Msg) at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:238
 in eventloop(::ZMQ.Socket) at /home/ubuntu/.julia/v0.5/IJulia/src/IJulia.jl:143
 in (::IJulia.##24#30)() at ./task.jl:309
while loading /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl, in expression starting on line 31
 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in utf8(::SubString{String}) at ./deprecated.jl:50
 in collect_to!(::Array{String,1}, ::Base.Generator{Array{SubString{String},1},Base.#utf8}, ::Int64, ::Int64) at ./array.jl:302
 in _collect(:

In [5]:
function normal_probs(logprobs::Vector)
    ret = copy(logprobs)
    max_lp = maximum(logprobs)
    ret.-=max_lp #Bring closer to zero
    map!(exp,ret)
    denom = sum(ret)
    ret./=denom
    ret
end
function weighted_average(logprobs, embeddings)
    ret = zeros(first(embeddings))
    for (weight, embedding) in zip(normal_probs(logprobs), embeddings)
        ret.+= weight.*embedding
    end
    ret
end

function all_logprobs_of_context(embed, context, middle_word)
    [Query.logprob_of_context(embed, context,wv; skip_oov=true, normalise_over_length=true)
        for wv in embed.embedding[middle_word]]
end

function synthesize_embedding(ee,context, middle_word)
    logprobs = all_logprobs_of_context(ee, context,middle_word)
    weighted_average(logprobs, ee.embedding[middle_word])
end

synthesize_embedding (generic function with 1 method)

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [6]:
function poormans_tokenize(source)
    cleaned = filter(s->!ispunct(s), lowercase(source))
    map(string,  split(cleaned))::Vector{SubString{String}}
end

function punctuation_space_tokenize(source)
    preprocced = lowercase(source)
    pass1=replace(preprocced,r"[[:punct:]]*[[:space:]][[:punct:]]*"," ")
    pass2=replace(pass1,r"[[:punct:]]*$|^[[:punct:]]*","")
    split(pass2)
end



punctuation_space_tokenize (generic function with 1 method)

In [7]:
function load_sense_key_map(wn_basedir)
    path=joinpath(wn_basedir, "dict","index.sense")
    sense_key_map = Dict{Tuple{Int64,String},String}()
    for line in eachline(path)
        full_sense_key, offset_str, sense_num_str, tagcount_str = split(line) 
        lemma_name = first(split(full_sense_key,'%'))
        sense_offset = parse(Int64, offset_str)
        index = (sense_offset,lemma_name)
        @assert(!haskey(sense_key_map, index))
        sense_key_map[index] = full_sense_key
    end
    sense_key_map
end


function sense_key(lem::Lemma, ss::Synset,sense_key_map)
    sense_key_map[(ss.offset,lem.word)]
end
function sense_keys(lem::Lemma,sense_key_map)
    [sense_key_map[(ss_offset,lem.word)] for ss_offset in lem.synset_offsets ]
end

sense_keys (generic function with 1 method)

In [8]:
sense_key_map = load_sense_key_map(WN_PATH);

In [11]:
sense_keys(db['a',"six"],sense_key_map)

1-element Array{String,1}:
 "six%5:00:00:cardinal:00"

In [12]:
sense_key_map[2134693,"discover"]

"discover%2:39:03::"

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [13]:
using LightXML

INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/LightXML.ji for module LightXML.
 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in @windows(::Any, ::Any) at ./deprecated.jl:502
 in include_from_node1(::String) at ./loading.jl:426
 in macro expansion; at ./none:2 [inlined]
 in anonymous at ./<missing>:?
 in eval(::Module, ::Any) at ./boot.jl:234
 in process_options(::Base.JLOptions) at ./client.jl:239
 in _start() at ./client.jl:318
while loading /home/ubuntu/.julia/v0.5/LightXML/src/LightXML.jl, in expression starting on line 7
  likely near /home/ubuntu/.julia/v0.5/LightXML/src/cdata.jl:1
  likely near /home/ubuntu/.julia/v0.5/LightXML/src/cdata.jl:6


In [17]:
immutable WsdChallenge
    id::String
    word::String
    lemma::String
    pos::Char
    context::Vector{String}
end

function load_challenges(xml_file="data/corpora/wsd/semeval2007_t7/test/eng-coarse-all-words.xml")
    xdoc = parse_file(xml_file)
    xroot = root(xdoc)
    Task() do
        for text_node in child_elements(xroot)
            #@show text_node
            #text = child_nodes(text_node) |> collect
            #println(text)
            for sentence_node in child_elements(text_node)
                sentence = punctuation_space_tokenize(content(sentence_node))

                for lemma_node in child_elements(sentence_node)
                    word = content(lemma_node) |> lowercase
                    lemma = attribute(lemma_node,"lemma")|> lowercase
                    pos = first(attribute(lemma_node,"pos"))
                    id = attribute(lemma_node,"id")
                    context = filter(w->w!=word, sentence)
                    produce(WsdChallenge(id, word, lemma, pos, context))
                end
            end
        end
    end
end
challenges = load_challenges() |> collect;

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in bytestring(::Ptr{UInt8}) at ./deprecated.jl:50
 in _xcopystr at /home/ubuntu/.julia/v0.5/LightXML/src/LightXML.jl:39 [inlined]
 in content(::LightXML.XMLNode) at /home/ubuntu/.julia/v0.5/LightXML/src/nodes.jl:171
 in (::##15#17{LightXML.XMLElement})() at ./In[17]:18
while loading In[17], in expression starting on line 32
 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in bytestring(::Ptr{UInt8}) at ./deprecated.jl:50
 in _xcopystr at /home/ubuntu/.julia/v0.5/LightXML/src/LightXML.jl:39 [inlined]
 in #attribute#1(::Bool, ::Function, ::LightXML.XMLElement, ::String) at /home/ubuntu/.julia/v0.5/LightXML/src/nodes.jl:226
 in (::##15#17{LightXML.XMLElement})() at ./In[17]:22
while loading In[17], in expression starting on line 32


In [18]:
immutable WsdSolution
    id
    lemma
    pos
    solutions
end
only_of_pos(data, pos) = filter(d->d.pos==pos, data)

function load_solutions(key_file="data/corpora/wsd/semeval2007_t7/key/dataset21.test.key")
    map(eachline(key_file)) do line
        line_data, comment = split(line,"!!")
        fields = split(line_data)
        doc_id = fields[1]
        instance_id = fields[2]
        solutions = fields[3:end]
        
        lemma,pos = match(r"lemma=(.*)#(.)", comment).captures
        WsdSolution(instance_id, lemma, pos[1], solutions)
    end
end

solutions = load_solutions();

In [19]:
challenge = challenges[1]

WsdChallenge("d001.s001.t001","editorial","editorial",'n',String["your","oct","6","the","ill","homeless","referred","to","research","by","us","and","six","of","our","colleagues","that","was","reported","in","the","sept","8","issue","of","the","journal","of","the","american","medical","association"])

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [20]:
function sense_frequency(ss::Synset, lem::Lemma)
    lem_count = lem.tagsense_count
    lem_count+0.1(sum(values(ss.word_counts))-lem_count)
end

sense_frequency (generic function with 1 method)

In [21]:
lexically_informed_embeddings(db,ee, lemma_word, pos) = lexically_informed_embeddings(db,ee, lemma_word, lemma_word, pos)

function lexically_informed_embeddings(db,ee, word,lemma_word, pos)
    source_wvs = ee.embedding[word]
    lemma = db[pos, lemma_word]
    target_synsets::Vector{Synset} = synsets(db, lemma)
    if length(source_wvs) == 1
        #Use MCWS
        score, index = findmax(sense_frequency(ss,lemma) for ss in target_synsets)
        target_synsets=[target_synsets[index]] # Throw out others
    end
            
    embeddings = Vector{eltype(source_wvs)}(length(target_synsets))
    for (ii,synset) in enumerate(target_synsets)
        context::Vector{SubString{String}} = collect(filter!(w->w!=word, punctuation_space_tokenize(synset.gloss)))
        embeddings[ii] = synthesize_embedding(ee,context, word)
    end
        
    embeddings,target_synsets,lemma
end

lexically_informed_embeddings (generic function with 2 methods)

In [22]:
methods(logprob_of_context)

# 2 methods for generic function "logprob_of_context":
logprob_of_context{S<:String}(embed::WordEmbeddings.WordEmbedding, context::AbstractArray{S,1}, middle_word::S; kwargs...) at /mnt_volume/phd/prototypes/SenseSplittingWord2Vec/src/Query.jl:142
logprob_of_context(embed::WordEmbeddings.GenWordEmbedding, context, input::Array{Float32,1}; skip_oov, normalise_over_length) at /mnt_volume/phd/prototypes/SenseSplittingWord2Vec/src/Query.jl:148

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [40]:
function supervised_wsd(challenge::WsdChallenge, ee::WordSenseEmbedding, wn::DB, sense_key_map)
    try
        embeddings,target_synsets,lemma = lexically_informed_embeddings(wn,ee,
                                                            challenge.word, 
                                                            challenge.lemma, challenge.pos)
        #sense_index = Query.WSD(ee, embeddings, challenge.context; skip_oov=true)
        logprobs_of_context = [logprob_of_context(ee,challenge.context, wv; skip_oov=true, normalise_over_length=true) 
                                    for wv in embeddings] 
        
        sense_freqs = 1# Float32[sense_frequency(ss,lemma)+1 for ss in target_synsets]
        sense_freqs./=sum(sense_freqs)
        
        sense_index= indmax(log(sense_freqs).+logprobs_of_context)

        
        synset = target_synsets[sense_index]
        sk = sense_key(lemma, synset, sense_key_map)
        Nullable(sk)
    catch ex
        isa(ex, KeyError) || rethrow(ex)
        Nullable{String}()
    end
end

function most_frequent_sense(challenge, wn::DB, sense_key_map)
    try
        lemma = wn[challenge.pos, challenge.lemma]
        target_synsets::Vector{Synset} = synsets(db, lemma)
        
        sense_freqs =  Float32[sense_frequency(ss,lemma) for ss in target_synsets]      
        sense_index= indmax(sense_freqs)
        synset = target_synsets[sense_index]
        
        sk = sense_key(lemma, synset, sense_key_map)
        Nullable(sk)
    catch ex
        isa(ex, KeyError) || rethrow(ex)
        Nullable{String}()
    end
end



most_frequent_sense (generic function with 3 methods)

In [41]:
function evalute_on_wsd_challenge(challenges, solutions, method)
    correct = 0
    incorrect = 0
    notattempted = 0
    for (challenge, ground_solution) in zip(challenges, solutions)
        assert(challenge.id == ground_solution.id)
        output_sense = method(challenge)
        if isnull(output_sense)
            notattempted+=1
        else
            if get(output_sense) ∈ ground_solution.solutions
                correct+=1
            else
                incorrect+=1
            end
        end
    end
    precision = correct/(correct+incorrect)
    recall = correct/(correct+incorrect+notattempted)
    f1 = (2*precision*recall) / (precision+recall)
    return precision, recall, f1
end
    
    
function evalute_on_wsd_challenge(challenges, solutions, ee::WordSenseEmbedding, wn::DB, sense_key_map)
    method = challenge -> supervised_wsd(challenge, ee, wn, sense_key_map)
    evalute_on_wsd_challenge(challenges, solutions, method)
end


function evalute_on_wsd_challenge_MFS(challenges, solutions, wn::DB, sense_key_map)
    method = challenge -> most_frequent_sense(challenge, wn, sense_key_map)
    evalute_on_wsd_challenge(challenges, solutions, method)
end

evalute_on_wsd_challenge_MFS (generic function with 1 method)

In [32]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, ee, db, sense_key_map))

println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), ee, db, sense_key_map))
println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), ee, db, sense_key_map))
println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), ee, db, sense_key_map))
println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), ee, db, sense_key_map))


overall: 		(0.6302250803858521,0.6046716615249008,0.6171839856050383)
only nouns  : 		(0.6470588235294118,0.6254512635379061,0.6360715924736118)
only verbs  : 		(0.5454545454545454,0.5177664974619289,0.53125)
only adjecti: 		(0.6960227272727273,0.6767955801104972,0.6862745098039216)
only adverbs: 		(0.6632124352331606,0.6153846153846154,0.6384039900249376)


In [42]:
println("overall: \t\t", evalute_on_wsd_challenge_MFS(challenges, solutions, db, sense_key_map))

println("only nouns  : \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), db, sense_key_map))
println("only verbs  : \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), db, sense_key_map))
println("only adjecti: \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), db, sense_key_map))
println("only adverbs: \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), db, sense_key_map))


overall: 		(0.5932128691053328,0.5932128691053328,0.5932128691053328)
only nouns  : 		(0.592057761732852,0.592057761732852,0.592057761732852)
only verbs  : 		(0.48392554991539766,0.48392554991539766,0.48392554991539766)
only adjecti: 		(0.6823204419889503,0.6823204419889503,0.6823204419889503)
only adverbs: 		(0.7548076923076923,0.7548076923076923,0.7548076923076922)
